# Spelling correction
## by
### Florian Eder    00819174
### Moritz Enderle  00819536

#### imports

In [1]:
import re
from collections import Counter

#### open txt

In [24]:
def words(_text):
    """
    Return all words in text
    :param text: text to be parsed
    :return: list of words
    """
    return re.findall(r'\w+', _text.lower())

with open('big.txt', encoding="utf-8") as f:
    text = f.read()
    WORDS = Counter(words(text))

#### given functions

In [3]:
def probability(_word, _no_words=sum(WORDS.values())):
    """
    Probability of `word`
    :param _word: word to be probed
    :param _no_words: number of words in the counter
    :return probability of `word`
    """
    return WORDS[_word] / _no_words


def correction(_word):
    """
    Most probable spelling correction for word
    :param _word: word to be corrected
    :return most probable spelling correction for word
    """
    return max(candidates(_word), key=probability)


def candidates(_word):
    """
    Generate possible spelling corrections for word
    :param _word: word to be corrected
    :return: list of possible spelling corrections for word
    """
    return known([_word]) or known(edits1(_word)) or known(edits2(_word)) or [_word]


def known(_words):
    """
    The subset of `words` that appear in the dictionary of WORDS
    :param _words: list of words to be checked
    :return: list of words that appear in the dictionary of WORDS
    """
    return set(w for w in _words if w in WORDS)


def edits1(_word):
    """All edits that are one edit away from `word`
    :param _word: word to be corrected
    :return: list of possible spelling corrections for word
    """
    letters = 'abcdefghijklmnopqrstuvwxyz'
    splits = [(_word[:i], _word[i:]) for i in range(len(_word) + 1)]
    deletes = [L + R[1:] for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
    replaces = [L + c + R[1:] for L, R in splits if R for c in letters]
    inserts = [L + c + R for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)


def edits2(_word):
    """
    All edits that are two edits away from `word`
    :param _word: word to be corrected
    :return: list of possible spelling corrections for word
    """
    return (e2 for e1 in edits1(_word) for e2 in edits1(e1))

#### improvements
#### context approach

In [25]:
%%time
CONTEXT = {}
text = text.strip().split()
for index, word in enumerate([x.strip() for x in text if x.isalpha()]):
    word = word.lower()
    if word not in CONTEXT:
        CONTEXT[word] = {}
    if index > 0:
        if text[index - 1] not in CONTEXT[word]:
            CONTEXT[word][text[index - 1]] = 0
        CONTEXT[word][text[index - 1]] += 1


CPU times: total: 93.8 ms
Wall time: 190 ms


In [12]:
def correction_context(_word, _context):
    _word = _word.lower()
    """
    Most probable spelling correction for word
    :param _word: word to be corrected
    :param _context: context of the word
    :return: most probable spelling correction for word
    """
    if _context == "":
        return max(candidates(_word), key=probability)
    else:
        return max(candidates(_word), key=lambda x: probability_context(x, _context))


def probability_context(_word, _context):
    """
    Probability of `word`
    :param _word: word to be probed
    :param _context: context of the word
    :return probability of `word`
    """
    if _word in CONTEXT and _context in CONTEXT[_word]:
        return CONTEXT[_word][_context] / sum(CONTEXT[_word].values())
    else:
        return 0

#### testing

In [6]:
from pprint import pprint

pprint(CONTEXT)

{'a': {'': 0,
       '(for': 1,
       '1000': 1,
       '30,000': 1,
       '4': 1,
       'A': 2,
       'ADLER.”': 1,
       'Ah,': 1,
       'All': 1,
       'And': 1,
       'Angel.”': 1,
       'At': 1,
       'Bachelor': 1,
       'Backwater,': 1,
       'Baker': 1,
       'Bakers': 1,
       'Bradstreet': 1,
       'British': 1,
       'But': 2,
       'But,': 1,
       'C.': 1,
       'City': 1,
       'Clark': 1,
       'Contents': 1,
       'Dockyard,': 1,
       'Doctor,”': 1,
       'Doran.': 1,
       'Dr.': 1,
       'Dundee': 1,
       'During': 1,
       'European': 1,
       'Eventually,': 1,
       'Every': 1,
       'FIVE': 1,
       'Florida': 1,
       'For': 2,
       'Get': 1,
       'God': 1,
       'Hatherley,': 1,
       'He': 9,
       'Helen!': 1,
       'Helen,’': 1,
       'Henry': 1,
       'He’s': 1,
       'His': 2,
       'Holmes': 11,
       'Holmes,': 3,
       'Holmes.': 1,
       'Horner': 1,
       'Hosmer': 2,
       'House': 1,
       'I': 73,


In [27]:
def spelltest(tests, verbose=False):
    "Run correction(wrong) on all (right, wrong) pairs; report results."
    import time
    start = time.time()
    good, unknown = 0, 0
    n = len(tests)
    for right, wrong in tests:
        w = correction(wrong)
        good += (w == right)
        if w != right:
            unknown += (right not in WORDS)
            if verbose:
                print('correction({}) => {} ({}); expected {} ({})'
                      .format(wrong, w, WORDS[w], right, WORDS[right]))
    dt = time.time() - start
    print('{:.0%} of {} correct ({:.0%} unknown) at {:.0f} words per second '
          .format(good / n, n, unknown / n, n / dt))

def testset(lines):
    "Parse 'right: wrong1 wrong2' lines into [('right', 'wrong1'), ('right', 'wrong2')] pairs."
    return [(right, wrong)
            for (right, wrongs) in (line.split(':') for line in lines)
            for wrong in wrongs.split()]

spelltest(testset(open('spell-testset1.txt'))) # Development set
spelltest(testset(open('spell-testset2.txt'))) # Final test set

correction(contenpted) => consented (1); expected contented (0)
correction(contende) => continue (9); expected contented (0)
correction(contended) => continued (14); expected contented (0)
correction(contentid) => contents (5); expected contented (0)
correction(proplen) => people (25); expected problem (20)
correction(exstacy) => eustace (1); expected ecstasy (0)
correction(ecstacy) => eustace (1); expected ecstasy (0)
correction(guic) => quick (25); expected juice (0)
correction(juce) => june (3); expected juice (0)
correction(jucie) => judge (4); expected juice (0)
correction(juise) => just (126); expected juice (0)
correction(juse) => just (126); expected juice (0)
correction(localy) => local (5); expected locally (0)
correction(compair) => company (18); expected compare (0)
correction(pronounciation) => pronounciation (0); expected pronunciation (0)
correction(transportibility) => transportibility (0); expected transportability (0)
correction(miniscule) => miniscule (0); expected m